In [1]:
# GEE Analysis with FastAPI Integration and MapStore Catalog
# This notebook demonstrates:
# 1. GEE Authentication using GCP service account
# 2. Creating cloudless Sentinel-2 imagery composite
# 3. Pushing results to FastAPI service
# 4. Making layers available for MapStore catalog

import ee
import json
import os
import sys
import requests
from datetime import datetime
import folium

# Add gee_lib to path
sys.path.insert(0, '/usr/src/app/gee_lib')

print("✓ Imports loaded successfully")

✓ Imports loaded successfully


In [2]:
# Step 1: Initialize Google Earth Engine with Service Account
def initialize_gee():
    """Initialize GEE using service account credentials"""
    try:
        # Load credentials from user_id.json
        credentials_path = '/usr/src/app/user_id.json'
        
        with open(credentials_path, 'r') as f:
            credentials_data = json.load(f)
        
        service_account = credentials_data['client_email']
        
        # Initialize Earth Engine
        credentials = ee.ServiceAccountCredentials(service_account, credentials_path)
        ee.Initialize(credentials)
        
        print(f"✓ GEE Initialized successfully")
        print(f"  Service Account: {service_account}")
        print(f"  Project ID: {credentials_data['project_id']}")
        
        return True
    except Exception as e:
        print(f"✗ Error initializing GEE: {e}")
        return False

# Initialize GEE
initialize_gee()


✓ GEE Initialized successfully
  Service Account: earth-engine-land-eligibility@ee-iwansetiawan.iam.gserviceaccount.com
  Project ID: ee-iwansetiawan


True

In [3]:
# Step 2: Define Area of Interest (AOI)
# Example: A region in Indonesia (Kalimantan)
aoi_coords = [
    [109.5, -1.5],
    [110.5, -1.5],
    [110.5, -0.5],
    [109.5, -0.5],
    [109.5, -1.5]
]

aoi = ee.Geometry.Polygon(aoi_coords)
aoi_center = aoi.centroid().coordinates().getInfo()

print("✓ AOI defined")
print(f"  Center coordinates: {aoi_center}")
print(f"  Area: {aoi.area().divide(1e6).getInfo():.2f} km²")


✓ AOI defined
  Center coordinates: [109.99999999999977, -1.000012676912049]
  Area: 12362.61 km²


In [4]:
# Step 3: Create Cloudless Sentinel-2 Composite using gee_lib
from osi.image_collection.main import ImageCollection

# Configuration
config = {
    'IsThermal': False,
}

date_start_end = ['2023-01-01', '2023-12-31']
cloud_cover_threshold = 20  # Maximum cloud cover percentage

print("Creating cloudless Sentinel-2 composite...")
print(f"  Date range: {date_start_end[0]} to {date_start_end[1]}")
print(f"  Cloud cover threshold: {cloud_cover_threshold}%")

# Create ImageCollection instance
img_collection = ImageCollection(
    I_satellite='Sentinel',
    region='asia',
    AOI=aoi,
    date_start_end=date_start_end,
    cloud_cover_threshold=cloud_cover_threshold,
    config=config
)

# Get cloud-masked image collection
sentinel_collection = img_collection.image_collection_mask()

# Create median composite (cloudless)
sentinel_composite = img_collection.image_mosaick()

print("✓ Sentinel-2 cloudless composite created")
print(f"  Number of images used: {sentinel_collection.size().getInfo()}")


Creating cloudless Sentinel-2 composite...
  Date range: 2023-01-01 to 2023-12-31
  Cloud cover threshold: 20%
selecting Sentinel images
selecting Sentinel images
✓ Sentinel-2 cloudless composite created
  Number of images used: 25


In [5]:
# Step 4: Generate Analysis Products
# Create True Color, False Color, and NDVI visualizations

# True Color RGB (Natural Color)
true_color = sentinel_composite.select(['red', 'green', 'blue'])

# False Color (NIR, Red, Green) - highlights vegetation
false_color = sentinel_composite.select(['nir', 'red', 'green'])

# Calculate NDVI
ndvi = sentinel_composite.normalizedDifference(['nir', 'red']).rename('NDVI')

# Calculate EVI (Enhanced Vegetation Index)
evi = sentinel_composite.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': sentinel_composite.select('nir'),
        'RED': sentinel_composite.select('red'),
        'BLUE': sentinel_composite.select('blue')
    }
).rename('EVI')

# Calculate NDWI (Normalized Difference Water Index)
ndwi = sentinel_composite.normalizedDifference(['green', 'nir']).rename('NDWI')

print("✓ Analysis products generated:")
print("  - True Color RGB")
print("  - False Color Composite")
print("  - NDVI (Normalized Difference Vegetation Index)")
print("  - EVI (Enhanced Vegetation Index)")
print("  - NDWI (Normalized Difference Water Index)")


✓ Analysis products generated:
  - True Color RGB
  - False Color Composite
  - NDVI (Normalized Difference Vegetation Index)
  - EVI (Enhanced Vegetation Index)
  - NDWI (Normalized Difference Water Index)


In [6]:
# Step 5: Create GEE Map IDs for Tile Serving
# These Map IDs can be used to serve tiles via URLs

# Visualization parameters
vis_params = {
    'true_color': {
        'bands': ['red', 'green', 'blue'],
        'min': 0,
        'max': 0.3,
        'gamma': 1.4
    },
    'false_color': {
        'bands': ['nir', 'red', 'green'],
        'min': 0,
        'max': 0.5,
        'gamma': 1.4
    },
    'ndvi': {
        'min': -0.2,
        'max': 0.8,
        'palette': ['red', 'yellow', 'green', 'darkgreen']
    },
    'evi': {
        'min': -0.2,
        'max': 0.8,
        'palette': ['brown', 'yellow', 'lightgreen', 'darkgreen']
    },
    'ndwi': {
        'min': -0.3,
        'max': 0.3,
        'palette': ['white', 'lightblue', 'blue', 'darkblue']
    }
}

# Generate Map IDs
print("Generating GEE Map IDs...")

map_ids = {}
map_ids['true_color'] = true_color.getMapId(vis_params['true_color'])
map_ids['false_color'] = false_color.getMapId(vis_params['false_color'])
map_ids['ndvi'] = ndvi.getMapId(vis_params['ndvi'])
map_ids['evi'] = evi.getMapId(vis_params['evi'])
map_ids['ndwi'] = ndwi.getMapId(vis_params['ndwi'])

print("✓ Map IDs generated for all layers")

# Display tile URLs
for layer_name, map_id_dict in map_ids.items():
    tile_url = map_id_dict['tile_fetcher'].url_format
    print(f"\n{layer_name.upper()}:")
    print(f"  Tile URL: {tile_url}")


Generating GEE Map IDs...
✓ Map IDs generated for all layers

TRUE_COLOR:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/669a8ebdfefb45ee3fc5181f482e1990-a9754b475d45e8d77a7c541ec47bda2d/tiles/{z}/{x}/{y}

FALSE_COLOR:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/6acf69e86f9504d9eb77ca36a3e53320-6002df6f94f531249cb35f7f58477599/tiles/{z}/{x}/{y}

NDVI:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/915cb21fce261d92847c2a842de9d0c7-a269ce1a46a1efcfde52bdba7d042455/tiles/{z}/{x}/{y}

EVI:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/667819738e9d58fb6589d4d3c3634463-5b030fbc88066175ea565213f7e4a80c/tiles/{z}/{x}/{y}

NDWI:
  Tile URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/ad0afcebc02f21ac5c89ecc41cfade40-0242eff98b738065342245ac01b8766a/tiles/{z}/{x}/{y}


In [7]:
# Step 6: Visualize on Interactive Map using Folium
# Create an interactive map centered on AOI

# Create base map
center_lat, center_lon = aoi_center[1], aoi_center[0]
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add GEE tile layers
for layer_name, map_id_dict in map_ids.items():
    tile_url = map_id_dict['tile_fetcher'].url_format
    folium.TileLayer(
        tiles=tile_url,
        attr='Google Earth Engine',
        name=layer_name.replace('_', ' ').title(),
        overlay=True,
        control=True
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

print("✓ Interactive map created with all layers")
print(f"  Map centered at: ({center_lat:.4f}, {center_lon:.4f})")

# Display the map
m


✓ Interactive map created with all layers
  Map centered at: (-1.0000, 110.0000)


In [8]:
# Step : Complete GEE-to-MapStore Integration (Simplified)
# This step uses the GEE Integration Library to handle all the complex processing

# import sys
# sys.path.append('/usr/src/app/fastapi-gee-service')
from gee_integration import process_gee_to_mapstore

map_layers = {}

layers_data = {'project_name': 'Sentinel-2 Cloudless Composite Analysis'}

# Add each layer from your GEE analysis
for layer_name, map_id_dict in map_ids.items():
    map_layers[layer_name] = {
        'tile_url': map_id_dict['tile_fetcher'].url_format,
        'name': layer_name.replace('_', ' ').title(),
        'description': f'{layer_name.upper()} visualization from GEE analysis',
        'vis_params': vis_params[layer_name]
    }

# Import the AOI fix function properly
from gee_lib.osi.utils import process_aoi_geometry

# Fix AOI coordinates using the existing function (same approach as notebook 1)
print(f"🔧 Fixing AOI coordinates...")
aoi_info = process_aoi_geometry(aoi)

print(f"✅ AOI processed successfully:")
print(f"   - Center: {aoi_info['center']}")
print(f"   - Area: Unknown")
print(f"   - BBox: {aoi_info['bbox']}")
print(f"✅ Fixed AOI Info:")
print(f"   Bbox: {aoi_info['bbox']}")
print(f"   Center: {aoi_info['center']}")

print("🚀 Processing GEE Analysis to MapStore...")
print(f"   Project: {layers_data['project_name']}")
print(f"   Layers: {len(map_layers)}")
print(f"   AOI: {aoi_info['bbox']}")

# Process the complete integration
result = process_gee_to_mapstore(
    map_layers=map_layers,
    project_name=layers_data['project_name'],
    aoi_info=aoi_info,
    clear_cache_first=False,
    fastapi_url="http://fastapi:8000"  # Internal Docker network
)


🔧 Fixing AOI coordinates...
✅ Centroid calculated successfully with error margin 1


INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/config/localConfig.json
INFO:gee_integration:Processing GEE analysis: Sentinel-2 Cloudless Composite Analysis
INFO:gee_integration:Using complex layer info for 'true_color': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'false_color': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'ndvi': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'evi': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Using complex layer info for 'ndwi': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Registering with FastAPI: sentinel_2_cloudless_composite_analysis_20251023_201656
INFO:gee_integration:✅ FastAPI registration successful: Map

Calculated bbox from coordinates: {'minx': 109.5, 'miny': -1.5, 'maxx': 110.5, 'maxy': -0.5}
✅ AOI processed successfully:
   - Center: [109.99999999999977, -1.000012676912049]
   - Area: 12362.61 km²
   - BBox: {'minx': 109.5, 'miny': -1.5, 'maxx': 110.5, 'maxy': -0.5}
✅ AOI processed successfully:
   - Center: [109.99999999999977, -1.000012676912049]
   - Area: Unknown
   - BBox: {'minx': 109.5, 'miny': -1.5, 'maxx': 110.5, 'maxy': -0.5}
✅ Fixed AOI Info:
   Bbox: {'minx': 109.5, 'miny': -1.5, 'maxx': 110.5, 'maxy': -0.5}
   Center: [109.99999999999977, -1.000012676912049]
🚀 Processing GEE Analysis to MapStore...
   Project: Sentinel-2 Cloudless Composite Analysis
   Layers: 5
   AOI: {'minx': 109.5, 'miny': -1.5, 'maxx': 110.5, 'maxy': -0.5}


INFO:gee_utils:Successfully updated MapStore WMTS service: gee_analysis_wmts
INFO:gee_utils:Getting current WMTS layers...
INFO:gee_utils:Comprehensive WMTS refresh completed successfully - Found 5 layers
INFO:gee_integration:✅ MapStore WMTS configuration updated
INFO:gee_integration:   New layers found: 5


In [ ]:
!cat /usr/src/app/mapstore/config/localConfig.json

In [ ]:
!cd .. && ls

In [ ]:
# Step 7: Prepare Data for FastAPI Service
# Create a payload with layer information and metadata

project_id = f"sentinel_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

# Prepare layer data for FastAPI
layers_data = {
    'project_id': project_id,
    'project_name': 'Sentinel-2 Cloudless Composite Analysis',
    'aoi': {
        'type': 'Polygon',
        'coordinates': [aoi_coords],
        'center': aoi_center
    },
    'date_range': {
        'start': date_start_end[0],
        'end': date_start_end[1]
    },
    'analysis_params': {
        'satellite': 'Sentinel-2',
        'cloud_cover_threshold': cloud_cover_threshold,
        'image_count': sentinel_collection.size().getInfo()
    },
    'layers': {}
}

# Add layer information
for layer_name, map_id_dict in map_ids.items():
    layers_data['layers'][layer_name] = {
        'name': layer_name.replace('_', ' ').title(),
        'description': f'{layer_name.upper()} visualization from Sentinel-2',
        'tile_url': map_id_dict['tile_fetcher'].url_format,
        'map_id': map_id_dict['mapid'],
        'token': map_id_dict['token'],
        'vis_params': vis_params[layer_name]
    }

print("✓ Data prepared for FastAPI")
print(f"  Project ID: {project_id}")
print(f"  Total layers: {len(layers_data['layers'])}")


In [ ]:
# Step 8: Push Results to FastAPI Service
# Send the analysis results to FastAPI so they can be served to MapStore

# FastAPI service URL (within Docker network)
fastapi_url = "http://fastapi:8000"

def push_to_fastapi(data, endpoint="/layers/register"):
    """Push GEE analysis results to FastAPI service"""
    try:
        # Send POST request to FastAPI
        url = f"{fastapi_url}{endpoint}"
        print(f"Pushing to: {url}")
        
        response = requests.post(
            url,
            json=data,
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✓ Successfully pushed to FastAPI")
            print(f"  Status: {result.get('status')}")
            print(f"  Project ID: {result.get('project_id')}")
            print(f"  Layers Count: {result.get('layers_count')}")
            print(f"  Message: {result.get('message')}")
            return result
        else:
            print(f"✗ Error: {response.status_code}")
            try:
                error_detail = response.json()
                print(f"  Detail: {error_detail.get('detail', response.text)}")
            except:
                print(f"  Response: {response.text}")
            return None
    except requests.exceptions.ConnectionError as e:
        print(f"✗ Connection Error: Cannot reach FastAPI service")
        print(f"  Make sure FastAPI container is running")
        print(f"  Check: docker ps | grep fastapi")
        return None
    except Exception as e:
        print(f"✗ Error pushing to FastAPI: {e}")
        import traceback
        print(f"  Traceback: {traceback.format_exc()}")
        return None

# Push data to FastAPI (using the new register endpoint)
print("Registering layers with FastAPI...")
result = push_to_fastapi(layers_data)

if result:
    print("\n📡 Data is now available in FastAPI service")
    print(f"   Access layers: {fastapi_url}/layers/{project_id}")
    print(f"   FastAPI Docs: {fastapi_url}/docs")
else:
    print("\n⚠️  Failed to push to FastAPI")
    print("   Layers are still available via direct GEE tile URLs")
    print("   You can still add them to MapStore manually")


In [ ]:
# Step 10: Test Direct Access to Layers
# Verify that the layers are accessible through FastAPI

def test_layer_access(project_id, layer_name):
    """Test if layer is accessible through FastAPI"""
    try:
        # Get layer information
        response = requests.get(
            f"{fastapi_url}/layers/{project_id}",
            timeout=10
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"✓ Layer accessible: {layer_name}")
            print(f"  Available layers: {list(result.get('layers', {}).keys())}")
            return True
        else:
            print(f"✗ Layer not accessible: {response.status_code}")
            return False
    except Exception as e:
        print(f"✗ Error testing access: {e}")
        return False

# Test access
print("Testing layer access through FastAPI...")
test_layer_access(project_id, 'ndvi')


In [ ]:
# Step 11: Update MapStore Catalog Dynamically
# This new approach automatically updates MapStore catalog with GEE results

# Import the catalog updater
import sys
sys.path.append('/usr/src/app/fastapi-gee-service')
from gee_catalog_updater import update_mapstore_catalog

# Prepare analysis information
analysis_params = {
    'satellite': 'Sentinel-2',
    'cloud_cover_threshold': cloud_cover_threshold,
    'image_count': sentinel_collection.size().getInfo(),
    'date_range': f"{date_start_end[0]} to {date_start_end[1]}"
}

aoi_info = {
    'center': aoi_center,
    'area_km2': aoi.area().divide(1e6).getInfo(),
    'coordinates': aoi_coords
}

print("🔄 Updating MapStore catalog with GEE results...")
print(f"  Project: {project_id}")
print(f"  Analysis: {analysis_params['satellite']} from {analysis_params['date_range']}")

# Update the catalog
catalog_result = update_mapstore_catalog(
    project_id=project_id,
    project_name=layers_data['project_name'],
    map_ids=map_ids,
    vis_params=vis_params,
    aoi_info=aoi_info,
    analysis_params=analysis_params,
    fastapi_url="http://fastapi:8000"
)

if catalog_result:
    print("\n✅ MapStore catalog updated successfully!")
    print("📋 Next steps:")
    print("   1. Go to MapStore: http://localhost:8082/mapstore")
    print("   2. Open the Catalog panel")
    print("   3. Look for 'GEE Dynamic Catalog' service")
    print("   4. Refresh the catalog to see your new layers")
    print("   5. Add layers to your map!")
    print(f"\n🔗 Catalog URL: {catalog_result.get('catalog_url')}")
else:
    print("\n❌ Failed to update MapStore catalog")
    print("   Check that FastAPI service is running")



In [ ]:
# Step 12: Manage and List Catalogs
# Check what catalogs are available and manage them

from gee_catalog_updater import GEECatalogUpdater

# Initialize catalog manager
catalog_manager = GEECatalogUpdater("http://fastapi:8000")

# List all available catalogs
print("📋 Available GEE Catalogs:")
catalogs = catalog_manager.list_catalogs()

if catalogs and catalogs.get('catalogs'):
    for catalog in catalogs['catalogs']:
        print(f"  • {catalog['project_name']} ({catalog['project_id']})")
        print(f"    Layers: {catalog['layers_count']}")
        print(f"    Updated: {catalog['timestamp']}")
        print(f"    Status: {catalog['status']}")
        print()
else:
    print("  No catalogs found")

# Get detailed info about current project
print(f"🔍 Detailed info for current project: {project_id}")
project_info = catalog_manager.get_catalog_info(project_id)

if project_info:
    print(f"  Project: {project_info['project_name']}")
    print(f"  Layers: {len(project_info['layers'])}")
    print("  Available layers:")
    for layer_name, layer_info in project_info['layers'].items():
        print(f"    • {layer_info['name']}: {layer_info['description']}")
    print(f"  Analysis info: {project_info['analysis_info']}")
else:
    print("  Project not found in catalog")

print("\n💡 Tips:")
print("  • Each time you run GEE analysis, it updates the catalog automatically")
print("  • New layers appear in MapStore after refreshing the catalog")
print("  • You can have multiple projects with different analysis results")
print("  • All layers are served as fast TMS tiles directly from GEE")


In [ ]:
# # example
# # In your Jupyter notebook, after generating GEE results:
# from gee_catalog_updater import update_mapstore_catalog

# result = update_mapstore_catalog(
#     project_id="my_analysis_20250101",
#     project_name="My GEE Analysis",
#     map_ids=map_ids,  # Your GEE map IDs
#     vis_params=vis_params,  # Your visualization parameters
#     aoi_info=aoi_info,  # Area of interest info
#     analysis_params=analysis_params  # Analysis parameters
# )

In [ ]:
# Step 13: Update MapStore WMTS Configuration Dynamically
# This step automatically updates the WMTS service in localConfig.json
# with the latest GEE analysis results

import sys
sys.path.append('/usr/src/app/fastapi-gee-service')
from wmts_config_updater import update_mapstore_wmts_config, get_current_wmts_status

# Prepare AOI information for WMTS configuration
aoi_bbox = {
    'minx': min([coord[0] for coord in aoi_coords]),
    'miny': min([coord[1] for coord in aoi_coords]),
    'maxx': max([coord[0] for coord in aoi_coords]),
    'maxy': max([coord[1] for coord in aoi_coords])
}

aoi_info_for_wmts = {
    'bbox': aoi_bbox,
    'center': aoi_center,
    'area_km2': aoi.area().divide(1e6).getInfo(),
    'coordinates': aoi_coords
}

print("🔄 Updating MapStore WMTS configuration...")
print(f"  Project: {project_id}")
print(f"  Analysis: {layers_data['project_name']}")
print(f"  AOI: {aoi_bbox}")

# Update WMTS configuration
wmts_success = update_mapstore_wmts_config(
    project_id=project_id,
    project_name=layers_data['project_name'],
    aoi_info=aoi_info_for_wmts
)

if wmts_success:
    print("\n✅ MapStore WMTS configuration updated successfully!")
    print("📋 WMTS Service Details:")
    print("   Service ID: GEE_analysis_WMTS_layers")
    print("   Type: WMTS (Web Map Tile Service)")
    print("   URL: http://localhost:8001/wmts")
    print("   Layers: Automatically discovered from WMTS GetCapabilities")
    print(f"   Project: {project_id}")
    
    # Show current status
    current_status = get_current_wmts_status()
    if current_status:
        print(f"\n🔍 Current WMTS Service Status:")
        print(f"   Service: {current_status['service_id']}")
        print(f"   Project: {current_status['project_name']}")
        print(f"   Generated: {current_status['generated_at']}")
        print(f"   Available Layers: {len(current_status['layers_available'])}")
        for layer in current_status['layers_available']:
            print(f"     • {layer}")
    
    print("\n📋 Next Steps:")
    print("   1. Go to MapStore: http://localhost:8082/mapstore")
    print("   2. Open the Catalog panel")
    print("   3. Look for 'GEE Analysis WMTS - [Project Name]' service")
    print("   4. Click on the service to see available layers")
    print("   5. Add individual layers to your map!")
    print("\n💡 Benefits of Dynamic WMTS:")
    print("   • Single WMTS service with multiple layers")
    print("   • Automatic layer discovery")
    print("   • No style mixing - each layer has proper styling")
    print("   • Easy to manage - old analyses are automatically replaced")
    
else:
    print("\n❌ Failed to update MapStore WMTS configuration")
    print("   Check that the MapStore config directory is accessible")
    print("   Verify Docker volume mount is working correctly")



In [ ]:
# Step 14: Manage WMTS Services
# Check current WMTS configuration and manage services

from wmts_config_updater import list_gee_services, get_current_wmts_status

print("🔍 Current GEE Services in MapStore Configuration:")
print("=" * 60)

# List all GEE services
gee_services = list_gee_services()

if gee_services:
    for i, service in enumerate(gee_services, 1):
        print(f"{i}. {service['service_id']}")
        print(f"   Type: {service['type']}")
        print(f"   Title: {service['title']}")
        if service['project_id']:
            print(f"   Project: {service['project_id']}")
        if service['generated_at']:
            print(f"   Generated: {service['generated_at']}")
        print()
else:
    print("   No GEE services found in configuration")

# Show current WMTS status
print("📊 Current WMTS Service Status:")
print("=" * 40)

current_wmts = get_current_wmts_status()
if current_wmts:
    print(f"✅ Active WMTS Service: {current_wmts['service_id']}")
    print(f"   Project: {current_wmts['project_name']}")
    print(f"   Project ID: {current_wmts['project_id']}")
    print(f"   Generated: {current_wmts['generated_at']}")
    print(f"   Available Layers ({len(current_wmts['layers_available'])}):")
    for layer in current_wmts['layers_available']:
        print(f"     • {layer}")
else:
    print("❌ No active WMTS service found")

print("\n💡 Management Tips:")
print("   • Each time you run GEE analysis, the WMTS service is automatically updated")
print("   • Old GEE services are automatically removed to prevent conflicts")
print("   • Only one WMTS service is active at a time (GEE_analysis_WMTS_layers)")
print("   • The WMTS service automatically discovers all available layers")
print("   • No manual configuration needed - everything is handled dynamically")


localhost:8888